Taller10 NLP - Daileny Dorado

In [1]:
import pandas as pd
import tweepy
import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
 
import matplotlib.pyplot as plt
%matplotlib inline
init_notebook_mode(connected=True)

## Punto 1

In [2]:
keys = pd.read_csv('keys_demo.csv', sep=';', header=None)
keys = dict(zip(keys[0],keys[1]))

In [3]:
auth = tweepy.OAuthHandler(keys['api_key'], keys['api_key_secret'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True)

In [7]:
tweets = tweepy.Cursor(api.search,
                       q=['Donald trump -RT'],
                       lang='es',
                       geocode=f'4.673495,-74.1316,45mi',
                       tweet_mode='extended',
                       include_rts=False).items(500)

In [8]:
t = []
 
for tweet in tweets:
    temp = []
    temp.append(tweet.full_text)
    temp.append(tweet.created_at)
    temp.append("Bogota")
    temp.append(tweet.user.location)
    temp.append(tweet.place)
    temp.append(tweet.coordinates)
    temp.append(tweet.id)
    temp.append(f"https://twitter.com/i/web/status/{tweet.id}")

    t.append(temp)

In [9]:
data = pd.DataFrame(t)
data.columns = ['tweet','date', 'city', 'user_location', 'place', 'coordinates', 'id', 'link']
data.head()

,tweet,date,city,user_location,place,coordinates,id,link
0,#AlAireEnLaFM Maria Paula Ochoa: Yo nunca hab...,2020-11-12 14:45:35,Bogota,"Bogotá, Colombia",None,None,1326898942060224513,https://twitter.com/i/web/status/1326898942060...
1,#AlAireEnLaFM Maria Paula Ochoa: En ese momen...,2020-11-12 14:44:36,Bogota,"Bogotá, Colombia",None,None,1326898698606047233,https://twitter.com/i/web/status/1326898698606...
2,@GabrielPeralBa Hillary Clinton concedió un dí...,2020-11-12 14:02:53,Bogota,"Bogotá, Colombia",None,None,1326888200242126850,https://twitter.com/i/web/status/1326888200242...
3,La máquina de mentiras de Donald Trump se ator...,2020-11-12 14:00:50,Bogota,Bogotá-Colombia,None,None,1326887681494818816,https://twitter.com/i/web/status/1326887681494...
4,La izquierda progresista y moralista tiene can...,2020-11-12 13:49:14,Bogota,"Bogotá, Colombia",None,None,1326884761252540419,https://twitter.com/i/web/status/1326884761252...


### Guardar archivo

In [10]:
data.to_csv('data.csv',index=False)

In [3]:
data = pd.read_csv('data.csv', sep=',')
data.head()

,tweet,date,city,user_location,place,coordinates,id,link
0,#AlAireEnLaFM Maria Paula Ochoa: Yo nunca hab...,2020-11-12 14:45:35,Bogota,"Bogotá, Colombia",NaN,NaN,1326898942060224513,https://twitter.com/i/web/status/1326898942060...
1,#AlAireEnLaFM Maria Paula Ochoa: En ese momen...,2020-11-12 14:44:36,Bogota,"Bogotá, Colombia",NaN,NaN,1326898698606047233,https://twitter.com/i/web/status/1326898698606...
2,@GabrielPeralBa Hillary Clinton concedió un dí...,2020-11-12 14:02:53,Bogota,"Bogotá, Colombia",NaN,NaN,1326888200242126850,https://twitter.com/i/web/status/1326888200242...
3,La máquina de mentiras de Donald Trump se ator...,2020-11-12 14:00:50,Bogota,Bogotá-Colombia,NaN,NaN,1326887681494818816,https://twitter.com/i/web/status/1326887681494...
4,La izquierda progresista y moralista tiene can...,2020-11-12 13:49:14,Bogota,"Bogotá, Colombia",NaN,NaN,1326884761252540419,https://twitter.com/i/web/status/1326884761252...


In [4]:
import re
 
pattern1 = '?P<pic>pic.twitter.com/[^\s]+'
pattern2 = '?P<url>https?://[^\s]+'
 
def text_clean(row):
    text = row['tweet']

    links = [tuple(j for j in i if j)[-1] for i in re.findall(f"({pattern1})|({pattern2})",text)]
    for link in links:
        text = text.replace(link,"")

    mentions = [interaction for interaction in text.split() if interaction.startswith("@")]
    for mention in mentions:
        text = text.replace(mention,"")

    return text, links, mentions

In [5]:
data[['clean', 'links', 'mentions']] = data.apply(text_clean, axis=1, result_type='expand')

data.head()

,tweet,date,city,user_location,place,coordinates,id,link,clean,links,mentions
0,#AlAireEnLaFM Maria Paula Ochoa: Yo nunca hab...,2020-11-12 14:45:35,Bogota,"Bogotá, Colombia",NaN,NaN,1326898942060224513,https://twitter.com/i/web/status/1326898942060...,#AlAireEnLaFM Maria Paula Ochoa: Yo nunca hab...,[https://t.co/eyTGBdxAeD],[]
1,#AlAireEnLaFM Maria Paula Ochoa: En ese momen...,2020-11-12 14:44:36,Bogota,"Bogotá, Colombia",NaN,NaN,1326898698606047233,https://twitter.com/i/web/status/1326898698606...,#AlAireEnLaFM Maria Paula Ochoa: En ese momen...,[https://t.co/eyTGBdxAeD],[]
2,@GabrielPeralBa Hillary Clinton concedió un dí...,2020-11-12 14:02:53,Bogota,"Bogotá, Colombia",NaN,NaN,1326888200242126850,https://twitter.com/i/web/status/1326888200242...,Hillary Clinton concedió un día después de la...,[https://t.co/Pn8AdZRB3I],[@GabrielPeralBa]
3,La máquina de mentiras de Donald Trump se ator...,2020-11-12 14:00:50,Bogota,Bogotá-Colombia,NaN,NaN,1326887681494818816,https://twitter.com/i/web/status/1326887681494...,La máquina de mentiras de Donald Trump se ator...,[https://t.co/2wUzKBPTuE],[]
4,La izquierda progresista y moralista tiene can...,2020-11-12 13:49:14,Bogota,"Bogotá, Colombia",NaN,NaN,1326884761252540419,https://twitter.com/i/web/status/1326884761252...,La izquierda progresista y moralista tiene can...,[https://t.co/A0ciFGofqK],[]


## Punto 2 - Visualización

In [6]:
data['date'] =  pd.to_datetime(data['date'])
data['fecha'] = data.date.dt.date
fecha_df = pd.DataFrame(data.fecha.value_counts()).reset_index()
fecha_df.columns = ['fecha', 'cnt']
fecha_df = fecha_df.sort_values('fecha')
fecha_df

,fecha,cnt
0,2020-11-07,191
1,2020-11-08,115
3,2020-11-09,60
2,2020-11-10,72
4,2020-11-11,41
5,2020-11-12,21


In [7]:
import plotly.graph_objs as go
fig=go.Figure(go.Scatter(x=fecha_df['fecha'],
                                y=fecha_df['cnt'],
                               mode='markers+lines',
                               name="Submissions",
                               marker_color='dodgerblue'))

fig.update_layout(
    title_text='Tweets por día : ({} - {}) '.format(fecha_df['fecha'].sort_values()[0].strftime("%d/%m/%Y"),
                                                       fecha_df['fecha'].sort_values().iloc[-1].strftime("%d/%m/%Y")),template="plotly_white",
    title_x=0.5)

fig.show()

Nota: 

Profe este taller lo realice con la ayuda de Alejandro Fandiño